## Deliverable 2. Create a Customer Travel Destinations Map.

In [27]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import os
import numpy as np

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [17]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv(os.path.join("..", "Weather_Database", "WeatherPy_database.csv"))
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Mahebourg,MU,-20.4081,57.7000,73.69,83,40,8.05,scattered clouds
1,1,Tevriz,RU,57.5090,72.4042,59.11,65,91,7.96,overcast clouds
2,2,Norman Wells,CA,65.2820,-126.8329,39.24,65,100,18.41,overcast clouds
3,3,Kaitangata,NZ,-46.2817,169.8464,53.08,71,67,5.01,broken clouds
4,4,Thompson,CA,55.7435,-97.8558,42.96,93,100,3.44,mist


In [18]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 

min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [20]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.

preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
7,7,Atuona,PF,-9.8000,-139.0333,78.31,79,10,17.45,light rain
8,8,Kapaa,US,22.0752,-159.3190,82.38,70,0,14.97,clear sky
17,17,Sharjah,AE,25.3573,55.4033,75.18,68,0,4.61,dust
19,19,Vaini,TO,-21.2000,-175.2000,75.36,69,75,11.50,broken clouds
22,22,Itarema,BR,-2.9248,-39.9167,77.95,90,56,10.63,broken clouds
23,23,Bilma,NE,18.6853,12.9164,77.70,25,0,12.68,clear sky
29,29,Porto Novo,BJ,6.4965,2.6036,82.22,85,56,4.83,broken clouds
30,30,Teluk Intan,MY,4.0259,101.0213,77.02,74,97,1.70,heavy intensity rain
31,31,Rikitea,PF,-23.1203,-134.9692,76.15,75,100,20.96,overcast clouds
40,40,Eyl,SO,7.9803,49.8164,82.85,68,9,17.67,clear sky


In [21]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                193
City                   193
Country                193
Lat                    193
Lng                    193
Max Temp               193
Humidity               193
Cloudiness             193
Wind Speed             193
Current Description    193
dtype: int64

In [23]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna(axis="index", how="any")
print(len(clean_df))
clean_df.head()

193


,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
7,7,Atuona,PF,-9.8000,-139.0333,78.31,79,10,17.45,light rain
8,8,Kapaa,US,22.0752,-159.3190,82.38,70,0,14.97,clear sky
17,17,Sharjah,AE,25.3573,55.4033,75.18,68,0,4.61,dust
19,19,Vaini,TO,-21.2000,-175.2000,75.36,69,75,11.50,broken clouds
22,22,Itarema,BR,-2.9248,-39.9167,77.95,90,56,10.63,broken clouds


In [24]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()


# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
7,Atuona,PF,78.31,light rain,-9.8000,-139.0333,
8,Kapaa,US,82.38,clear sky,22.0752,-159.3190,
17,Sharjah,AE,75.18,dust,25.3573,55.4033,
19,Vaini,TO,75.36,broken clouds,-21.2000,-175.2000,
22,Itarema,BR,77.95,broken clouds,-2.9248,-39.9167,
23,Bilma,NE,77.70,clear sky,18.6853,12.9164,
29,Porto Novo,BJ,82.22,broken clouds,6.4965,2.6036,
30,Teluk Intan,MY,77.02,heavy intensity rain,4.0259,101.0213,
31,Rikitea,PF,76.15,overcast clouds,-23.1203,-134.9692,
40,Eyl,SO,82.85,clear sky,7.9803,49.8164,


In [28]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found...skipping the city.")
        hotel_df.loc[index, "Hotel Name"] = np.nan

Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.


In [29]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna(axis="index", how="any")
clean_hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
7,Atuona,PF,78.31,light rain,-9.8000,-139.0333,Villa Enata
8,Kapaa,US,82.38,clear sky,22.0752,-159.3190,Sheraton Kauai Coconut Beach Resort
17,Sharjah,AE,75.18,dust,25.3573,55.4033,Swiss Belhotel Sharjah
19,Vaini,TO,75.36,broken clouds,-21.2000,-175.2000,Keleti Beach Resort
22,Itarema,BR,77.95,broken clouds,-2.9248,-39.9167,"Sea Lounge! Bar, Club & Restaurant"
23,Bilma,NE,77.70,clear sky,18.6853,12.9164,Expeditions Ténére voyages
29,Porto Novo,BJ,82.22,broken clouds,6.4965,2.6036,Tour Eiffel Hotel Benin
30,Teluk Intan,MY,77.02,heavy intensity rain,4.0259,101.0213,Angsoka Hotel Teluk Intan
31,Rikitea,PF,76.15,overcast clouds,-23.1203,-134.9692,People ThankYou
40,Eyl,SO,82.85,clear sky,7.9803,49.8164,Amiin Hotel


In [30]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [31]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [32]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(markers)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))